In [17]:
import os

from ladybug.sql import SQLiteResult
from ladybug import analysisperiod as ap

from eppy import *

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import pandas as pd

import numpy as np

from collections import OrderedDict

import datetime as dt

from IPython.display import display



# Class Definition

In [18]:
class calibration_functions():
    def __init__(self):
        self.root = os.getcwd()
        self.hist_month_by_hr = None
        self.hist_monthly = None
        self.sim_data_hr = None
        self.sim_data_month = None
        self.sim_sqld = None
        self.hours = [dt.datetime(2019, 1, 1, i ).strftime("%H:%M") for i in range(24)]
        self.sim_folder = None
        

    def monthly_by_hour_from_sql(self, sql_dir_name):
        sqlFile = os.path.join(sql_dir_name, "eplusout.sql")
        sqld = SQLiteResult(sqlFile)

        elect = sqld.data_collections_by_output_name("Electricity:Facility")[-1]
        hot = sqld.data_collections_by_output_name("DistrictHeating:Facility")[-1]
        chill = sqld.data_collections_by_output_name("DistrictCooling:Facility")[-1]

        sim_month_by_hr = OrderedDict({
        "elect": elect.average_monthly_per_hour(),
        "hot": hot.average_monthly_per_hour(),
        "chill": chill.average_monthly_per_hour()
        })

        sim_monthly= OrderedDict({
        "elect": elect.total_monthly(),
        "hot": hot.total_monthly(),
        "chill": chill.total_monthly(),
        })

        return sim_month_by_hr, sim_monthly, sqld
    
    
    def load_simulations(self, sim_names, eppy_adjusted_sim_folder):
        "CALLABLE!"
        
        adjusted_model_dir = os.path.join(self.root, eppy_adjusted_sim_folder)
        self.sim_folder = adjusted_model_dir
        sim_name_paths = {k: os.path.join(adjusted_model_dir, v) for k,v in sim_names.items()}

        sim_data_hr = OrderedDict()
        sim_data_month = OrderedDict()
        sim_sqld = OrderedDict()

        for k, v in zip(sim_names.values(), sim_name_paths.values()):
            sim_data_hr[k] = self.monthly_by_hour_from_sql(v)[0]
            sim_data_month[k] = self.monthly_by_hour_from_sql(v)[1]
            sim_sqld[k] = self.monthly_by_hour_from_sql(v)[2]
        self.sim_data_hr = sim_data_hr
        self.sim_data_month = sim_data_month
        self.sim_sqld = sim_sqld

        # return sim_data_hr, sim_data_month 
    
    
    def plot_output_by_zone(self, sim_key, data_name, month_num):
        sqld = self.sim_sqld[sim_key]
        sql_outputs_info_0 = sqld.available_outputs_info
        sql_outputs_info = {item["output_name"]: {"name": item["output_name"], 
                            "units": item["units"],
                            "data_type": item["data_type"]} 
                            for item in sql_outputs_info_0 }
        zone_data = sqld.data_collections_by_output_name_run_period(data_name, 7)
        zonal_data  = OrderedDict({ix: value.average_monthly_per_hour() for ix, value in enumerate(zone_data)})
        num_zones = len(list(zonal_data.keys()))

        month = ap.AnalysisPeriod(st_month=month_num, end_month=month_num)

        data_type = sql_outputs_info[data_name]["data_type"]
        units = sql_outputs_info[data_name]["units"]

        fig = go.Figure()

        for k, v in zonal_data.items():
                fig.add_trace(
                    go.Scatter(x=self.hours, y=v.filter_by_analysis_period(month).values, mode='lines+markers', marker_size=num_zones*7-5*(k+1), name=f"Zone {k+1}")
                )

        fig.update_layout(
            title=f"{data_name} Month #{month_num}",
            xaxis_title="Hours",
            yaxis_title=f"{data_type} [{units}]",
        )
        fig.show()

    def plot_system_sizing_data(self, sim_key):
        ddySizeFile = os.path.join(self.sim_folder, sim_key, "epluszsz.csv")
        des_loads = pd.read_csv(ddySizeFile)
        fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=("Heat Mass Flow [kg/s]", 
            "Heat Load [W]", "Cool Load [W]"))


        for i in [1,2]:
            for k in [1,3]:
                fig.add_trace(
                            go.Scatter(x=des_loads.Time, 
                            y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s]"], mode='lines+markers', 
                            name=f"Heating Space {i}0{k}",
                            ),row=1, col=1)
        for i in [1,2]:
            for k in [1,3]:
                fig.add_trace(
                            go.Scatter(x=des_loads.Time, 
                            y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN HTG 99.6% CONDNS DB:Des Heat Load [W]"], mode='lines+markers', 
                            name=f"Heating Space {i}0{k}"
                            ), row=1, col=2)

        for i in [1,2]:
            for k in [1,3]:
                fig.add_trace(
                            go.Scatter(x=des_loads.Time, 
                            y=des_loads[f"THERMAL ZONE: SPACE {i}0{k}:CA_PALO-ALTO-AP ANN CLG 0.4% CONDNS DB=>MCWB:Des Sens Cool Load [W]"], mode='lines+markers', 
                            name=f"Cooling Space {i}0{k}"
                            ), row=1, col=3)

        fig.show()


        



    def calc_historical_data(self):
        # conversion factors
        tonHr_kwHr = 3.5169 # 1 ton-hr = 3.517 kWh
        kbtu_kwHr = 1/3.412 # 1 kbtu = 0.29 kwHr

        # load data 
        historical_data_path = os.path.join(self.root, "data/Thornton Utilities 2019.xlsx")

        # electricity # kwh 
        hist_elect_hourly = pd.read_excel(historical_data_path, sheet_name="Electricity hourly", usecols="A:E")

        # chilled water  -> convert ton hours to kWh
        hist_chw_hourly = pd.read_excel(historical_data_path, sheet_name="CHW hourly", usecols="A:E")
        hist_chw_hourly["kWh"] = hist_chw_hourly["ton-hours"]*tonHr_kwHr

        # hot water -> convert kbtu to kWh
        hist_hw_hourly = pd.read_excel(historical_data_path, sheet_name="HW hourly", usecols="A:E")
        hist_hw_hourly["kWh"] = hist_hw_hourly["kBtu"]*kbtu_kwHr 

        # month by hour
        self.hist_month_by_hr = OrderedDict({
            "elect" : hist_elect_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
            "hot": hist_hw_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
            "chill": hist_chw_hourly.groupby(["Month", "Hour"])["kWh"].mean(),
        })

        # monthly 
        self.hist_monthly = OrderedDict({
        "elect": hist_elect_hourly.groupby("Month")["kWh"].sum(),
        "hot":  hist_hw_hourly.groupby("Month")["kWh"].sum(),
        "chill":  hist_chw_hourly.groupby("Month")["kWh"].sum()
        })

    def calc_metrics(self, simulated, measured):
        ts = simulated
        tm = measured
        tm_avg = np.mean(measured)
        n = len(tm)
        rmse = (100/tm_avg) * np.sqrt( (1/n) * np.sum((ts - tm)**2) )
        mbe = (100/tm_avg) * (np.sum(ts - tm)/n)
        return np.round(rmse,3), np.round(mbe,3)

    def plot_datasets_monthly_by_hour(self, month_num, month_name):
        """
        month_num = integer number 1 - 12
        month_name = string month name 
        """
        hours = [dt.datetime(2019, 1, 1, i ).strftime("%H:%M") for i in range(24)]

        month = ap.AnalysisPeriod(st_month=month_num, end_month=month_num)
        blueShades = px.colors.qualitative.Vivid[:len(self.sim_data_hr.keys())]

        fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=("Electricity", "Hot Water", "Chilled Water"))

        # historical/ measured data 
        for (ix, value) in  enumerate(self.hist_month_by_hr.values()):
            showLegend = True
            if ix > 0:
                showLegend = False
            fig.add_trace(go.Scatter(  
                x=hours,
                y=value[month.st_month] , 
                mode='lines',
                name="Historical",
                legendgroup="Months",
                line=dict(color='red'),
                showlegend=showLegend),
                row=1, col=ix+1)

        # simulated datasets 
        for (name, dataset), blueShade in zip(self.sim_data_hr.items(), blueShades):
            for (ix, sim), hist in zip(enumerate(dataset.values()),self.hist_month_by_hr.values()) :
                showLegend = True
                if ix > 0:
                    showLegend = False
                fig.add_trace(go.Scatter( 
                    x=hours,
                    y=sim.filter_by_analysis_period(month).values , 
                    mode='lines+markers',
                    name=name,
                    legendgroup="Months",
                    line=dict(color=blueShade),
                    showlegend=showLegend),
                    row=1, col=ix+1)

        fig.update_layout(title=f"{month_name} Average Hourly Energy Usage",
            yaxis_title="Energy (kWh)",)
    
    # calculate metrics 
        metrics = {}
        for name, dataset in self.sim_data_hr.items():
            type_dict = {}
            for (k, sim), hist in zip(dataset.items(), self.hist_month_by_hr.values()):
                type_dict[k] = self.calc_metrics(sim.filter_by_analysis_period(month).values, hist[month.st_month])
            metrics[name] = type_dict
        metrics = pd.DataFrame(metrics)
        display(metrics)

        fig.show()
    

    def plot_datasets_monthly(self):
        months = [dt.datetime(2019, i+1, 1).strftime("%b") for i in range(12)]
        blueShades = px.colors.qualitative.Vivid[:len(self.sim_data_month.keys())]

        fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=("Electricity", "Hot Water", "Chilled Water"))

        # historical/ measured data 
        for (ix, value) in  enumerate(self.hist_monthly.values()):
            showLegend = True
            if ix > 0:
                showLegend = False
            fig.add_trace(go.Scatter(  
                x=months,
                y=value, 
                mode='lines',
                name="Historical",
                legendgroup="Months",
                line=dict(color='red'),
                showlegend=showLegend),
                row=1, col=ix+1)

        # simulated datasets 
        for (name, dataset), blueShade in zip(self.sim_data_month.items(), blueShades):
            for (ix, sim), hist in zip(enumerate(dataset.values()),self.hist_monthly.values()) :
                showLegend = True
                if ix > 0:
                    showLegend = False
                fig.add_trace(go.Scatter( 
                    x=months,
                    y=sim.values , 
                    mode='lines+markers',
                    name=name,
                    legendgroup="Months",
                    line=dict(color=blueShade),
                    showlegend=showLegend),
                    row=1, col=ix+1)

        fig.update_layout(title=f"Monthly Total Energy Usage",
            yaxis_title="Energy (kWh)",)
    
    # calculate metrics 
        metrics = {}
        for name, dataset in self.sim_data_month.items():
            type_dict = {}
            for (k, sim), hist in zip(dataset.items(), self.hist_monthly.values()):
                type_dict[k] = self.calc_metrics(sim.values, hist)
            metrics[name] = type_dict
        metrics = pd.DataFrame(metrics)
        display(metrics)

        fig.show()

        return fig

# Test

In [19]:
# initialize class
c = calibration_functions()
# load historical data
c.calc_historical_data()

sim_names = {
0: "eh6",
1: "03_09_sizing_zone_max_af_fraction_0.8",
2: "sac_ddy",
21: "eh8"
}
eppy_adjusted_sim_folder = "eppy_adjusted_models_2"


# pass in desired set of simulations 
c.load_simulations(sim_names, eppy_adjusted_sim_folder)

In [20]:
c.plot_datasets_monthly()

,eh6,03_09_sizing_zone_max_af_fraction_0.8,sac_ddy,eh8
elect,"(34.158, -32.478)","(34.158, -32.478)","(34.158, -32.478)","(34.331, -32.772)"
hot,"(84.888, -74.878)","(84.888, -74.878)","(84.888, -74.878)","(49.623, -39.36)"
chill,"(68.056, 51.115)","(68.056, 51.115)","(68.056, 51.115)","(39.435, 24.731)"


In [21]:
month = 8 
c.plot_output_by_zone(sim_names[21], 'Zone Total Internal Total Heating Rate', month)
c.plot_output_by_zone(sim_names[21], 'Zone Lights Total Heating Rate', month)
c.plot_output_by_zone(sim_names[21], 'Zone Electric Equipment Total Heating Rate', month)
c.plot_output_by_zone(sim_names[21], 'Zone People Total Heating Rate', month)
c.plot_output_by_zone(sim_names[21], "Zone People Total Heating Energy", month) # energy ~ 1/1000 * power
print("building envelope heat gains ")
c.plot_output_by_zone(sim_names[21], 'Zone Windows Total Heat Gain Energy', month)
c.plot_output_by_zone(sim_names[21], 'Surface Heat Storage Energy', month)
c.plot_output_by_zone(sim_names[21], 'Zone Air Heat Balance Surface Convection Rate', month)

building envelope heat gains 


In [22]:
c.plot_output_by_zone("eh6", "Zone People Total Heating Energy", 1)

In [23]:
c.plot_output_by_zone("eh8", "Zone People Total Heating Energy", 1)

In [24]:
c.plot_system_sizing_data("sac_ddy")